<a href="https://colab.research.google.com/github/dcarretero433-png/Modelo-IA-Perros-Gatos/blob/main/Perro_Gato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CARGAR EL MODELO**

Este código activa el modo legacy de Keras para asegurar compatibilidad, importa Keras desde TensorFlow y luego carga un modelo previamente entrenado (en este caso, el archivo keras_model.h5 generado en Teachable Machine). También abre y lee el archivo labels.txt, que contiene los nombres de las clases del modelo (“Perro”, “Gato”), y finalmente imprime esa lista de etiquetas.

In [ ]:
import os
# 1. Activamos el modo legacy (Keras 2)
os.environ['TF_USE_LEGACY_KERAS'] = '1'

from tensorflow import keras

#Carga el modelo
model = keras.models.load_model("/content/keras_model.h5", compile=False)

# Carga las etiquetas de las clases
nombre_clases = open("/content/labels.txt", "r").readlines()

print(nombre_clases)

['0 Gato\n', '1 Perro\n']


**ADQUIRIR DATOS NUEVOS**

Este fragmento usa la librería Pillow (PIL) para abrir una imagen llamada cat.983.jpg y convertirla al formato RGB, asegurando que tenga los tres canales de color necesarios para que el modelo de clasificación pueda procesarla correctamente.

In [ ]:
from PIL import Image, ImageOps

#Convertimos la imagen JPG a RGB
imagen= Image.open("cat.983.jpg").convert("RGB")

**PRE-PROCESAMIENTO**

Este código ajusta la imagen a un tamaño de 224×224 píxeles usando un reescalado de alta calidad, la convierte a un array NumPy para que pueda ser usada por el modelo, y luego normaliza sus valores de color llevándolos al rango [-1, 1], que es el formato esperado por los modelos exportados desde Teachable Machine.

In [ ]:
size = (224, 224)
imagen = ImageOps.fit(imagen, size, Image.Resampling.LANCZOS)

import numpy as np

# Convertimos la imagen en un array NumPy.
imagen_array = np.asarray(imagen)

normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1

**PREDICCIÓN**

Este código crea un arreglo (batch) de una sola imagen con la forma que el modelo necesita (1, 224, 224, 3), coloca dentro de él la imagen previamente normalizada y luego usa model.predict() para obtener las predicciones del modelo, es decir, la probabilidad de que la imagen sea un perro o un gato.

In [ ]:
# Crear un array para un lote de 1 imagen. ndarray = N-Dimensional Array
lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

lote_imagenes[0] = normalizada_imagen_array

resultados= model.predict(lote_imagenes)

1/1 [==============================] - 1s 1s/step


**POST-PROCESAMIENTO**

Este código toma las probabilidades generadas por el modelo, obtiene el índice de la clase con mayor probabilidad usando argmax, selecciona la etiqueta correspondiente desde la lista de nombres de clases y la imprime; luego muestra también la probabilidad asociada a esa predicción, indicando qué tan seguro está el modelo de que la imagen pertenece a esa clase.

In [ ]:
indice = np.argmax(resultados[0])

etiqueta = nombre_clases[indice]
print("La imagen es de clase: ", etiqueta)

probabilidad = resultados[0][indice]
print("Con una probabilidad de: ", probabilidad)


La imagen es de clase:  0 Gato

Con una probabilidad de:  0.9999808


**PREPARACIÓN DEL LOTE Y REFACTORIZACIÓN**

Este código define una función llamada predecir_imagen que recibe la ruta de una imagen, la abre y la convierte a RGB, la redimensiona a 224×224 píxeles, la transforma en un array NumPy y la normaliza al rango [-1, 1] para prepararla para el modelo; luego crea un lote con una sola imagen, lo pasa al modelo para obtener las predicciones, identifica la clase con mayor probabilidad y finalmente devuelve la etiqueta predicha y su probabilidad.

In [ ]:
from PIL import Image, ImageOps
import numpy as np
import os

def predecir_imagen(ruta_imagen):

    imagen = Image.open(ruta_imagen).convert("RGB")

    size = (224, 224)
    imagen = ImageOps.fit(imagen, size, Image.Resampling.LANCZOS)
    imagen_array = np.asarray(imagen)
    normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1

    lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    lote_imagenes[0] = normalizada_imagen_array
    resultados= model.predict(lote_imagenes, verbose=0)

    indice = np.argmax(resultados[0])
    etiqueta_predicha = nombre_clases[indice]
    probabilidad = resultados[0][indice]

    return etiqueta_predicha, probabilidad


Este código recorre todas las imágenes de una carpeta de prueba, determina si cada archivo debería ser “Gato” o “Perro” según su nombre, y luego llama a la función predecir_imagen para obtener la etiqueta y probabilidad que devuelve el modelo; con esto va contando cuántas predicciones totales se hacen, cuántas son correctas y acumulando la probabilidad de los aciertos, además de guardar en una lista la información de cada imagen mal clasificada (archivo, predicción y probabilidad). Al final imprime el número total de predicciones, cuántas fueron correctas, los detalles de los errores y la media de probabilidad de las predicciones acertadas.

In [ ]:
lista_archivos = os.listdir("/content/drive/MyDrive/cats_dogs_light/test")

total_predicciones = 0
aciertos = 0
media_probabilidad_aciertos = 0.0
# predicciones incorrectas contendrá una lista con los nombres de las imágenes mal clasificadas
predicciones_incorrectas = []

for nombre_archivo in lista_archivos:
    if "cat" in nombre_archivo:
        etiqueta_esperada = "0 Gato\n"
    elif "dog" in nombre_archivo:
        etiqueta_esperada = "1 Perro\n"
    else:
        continue  # Saltar archivos que no sean de gatos o perros

    total_predicciones += 1

    ruta_imagen = os.path.join("/content/drive/MyDrive/cats_dogs_light/test", nombre_archivo)

    # Aquí obtendriamos la predicción del modelo llamando a una función que
    # implemente las fases de inferencia
    etiqueta_predicha, probabilidad = predecir_imagen(ruta_imagen)

    if etiqueta_predicha == etiqueta_esperada:
        aciertos += 1
        media_probabilidad_aciertos += probabilidad
    else:
        info_error= {"archivo": nombre_archivo, "prediccion": etiqueta_predicha,
             "probabilidad": probabilidad}
        predicciones_incorrectas.append(info_error)

print(f"El total de predicciones es: {total_predicciones}")
print(f"El total de aciertos son: {aciertos}")
print(f"La información de las predicciones incorrectas es: {predicciones_incorrectas}")
print(f"La media de probabilidad de aciertos es: {media_probabilidad_aciertos}")


El total de predicciones es: 400
El total de aciertos son: 387
La información de las predicciones incorrectas es: [{'archivo': 'dog.9875.jpg', 'prediccion': '0 Gato\n', 'probabilidad': np.float32(0.6057579)}, {'archivo': 'dog.9911.jpg', 'prediccion': '0 Gato\n', 'probabilidad': np.float32(0.944507)}, {'archivo': 'dog.9868.jpg', 'prediccion': '0 Gato\n', 'probabilidad': np.float32(0.6471474)}, {'archivo': 'dog.9935.jpg', 'prediccion': '0 Gato\n', 'probabilidad': np.float32(0.9063609)}, {'archivo': 'cat.9836.jpg', 'prediccion': '1 Perro\n', 'probabilidad': np.float32(0.73044497)}, {'archivo': 'cat.9874.jpg', 'prediccion': '1 Perro\n', 'probabilidad': np.float32(0.6532888)}, {'archivo': 'cat.9897.jpg', 'prediccion': '1 Perro\n', 'probabilidad': np.float32(0.7792187)}, {'archivo': 'cat.9882.jpg', 'prediccion': '1 Perro\n', 'probabilidad': np.float32(0.9271227)}, {'archivo': 'cat.9960.jpg', 'prediccion': '1 Perro\n', 'probabilidad': np.float32(0.9742766)}, {'archivo': 'cat.9954.jpg', 'predi

**CÁLCULO DE MÉTRICAS Y GENERACIÓN DEL INFORME**

Este código calcula la precisión del modelo dividiendo los aciertos entre el total de predicciones, obtiene la probabilidad media de los aciertos, y construye un informe de evaluación en formato de texto que resume el rendimiento: cuántas predicciones se hicieron, cuántas fueron correctas, la precisión en porcentaje, la probabilidad media de aciertos y una lista detallada de las imágenes mal clasificadas con la predicción realizada y su probabilidad. Finalmente imprime el informe completo.

In [ ]:
precision = aciertos / total_predicciones if total_predicciones > 0 else 0
media_probabilidad_aciertos /= aciertos if aciertos > 0 else 1

informe = f"""
Informe de Evaluación del Modelo

Total de predicciones: {total_predicciones}
Aciertos: {aciertos}
Precisión: {precision:.2%}
Probabilidad media de aciertos: {media_probabilidad_aciertos:.2%}

Predicciones incorrectas:
"""

for error in predicciones_incorrectas:
    informe += f"Archivo: {error['archivo']}, Predicción: {error['prediccion'].strip()}, Probabilidad: {error['probabilidad']:.2%}\n"

print(informe)

NameError: name 'total_predicciones' is not defined